In [1]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
ls ../results/muestras

gaussian_1_1e6/  kraskov_1/  kraskov_2/


In [3]:
ls ../results/firing_rates

dmn/


In [4]:
is_dmn = True

In [5]:
if is_dmn:
    results_folder = "../results/firing_rates/"
else:
    results_folder = "../results/muestras/kraskov_2/"

gaussian_results = {}
for gaussian_i in sorted(os.listdir(results_folder)):
    gaussian_i_folder = os.path.join(results_folder, gaussian_i)
    gaussian_results[gaussian_i] = {
        "mean": [],
        "std": []
    }
    for n_data in sorted(os.listdir(gaussian_i_folder)):
        data_filepath = os.path.join(gaussian_i_folder, n_data, "results_samples.txt")
        data = np.loadtxt(data_filepath, delimiter=",")
        print(gaussian_i, n_data, data.shape, data_filepath)
        data_mean, data_std = data.mean(axis=0), data.std(axis=0)
        gaussian_results[gaussian_i]["mean"].append(data_mean)
        gaussian_results[gaussian_i]["std"].append(data_std)

dmn 100 (100, 3) ../results/firing_rates/dmn/100/results_samples.txt
dmn 1000 (100, 3) ../results/firing_rates/dmn/1000/results_samples.txt
dmn 10000 (100, 3) ../results/firing_rates/dmn/10000/results_samples.txt
dmn 100000 (100, 3) ../results/firing_rates/dmn/100000/results_samples.txt


OSError: ../results/firing_rates/dmn/1000000/results_samples.txt not found.

In [ ]:
gaussian_results

In [ ]:
try:
    with open("../data/muestras_1e5/index.json", "r") as f:
        index = json.load(f)
except:
    pass

In [ ]:
index

In [ ]:
metrics_plot = ["TC", "DTC", "O"]
#index_data = [100, 1000, 10000, 100000, 1000000]
index_data = [100, 1000, 10000, 100000]
for gaussian_i in sorted(os.listdir(results_folder)):
    try:
        fig, ax = plt.subplots(figsize=(10,5))
        mean = pd.DataFrame(gaussian_results[gaussian_i]["mean"], columns=["TC", "DTC", "O"], index=index_data)
        std = pd.DataFrame(gaussian_results[gaussian_i]["std"], columns=["TC", "DTC", "O"], index=index_data)
        mean[metrics_plot].plot(ax=ax)
        for i, metric_plot in enumerate(metrics_plot):
            ax.fill_between(mean.index, mean[metric_plot]-3*std[metric_plot], mean[metric_plot]+3*std[metric_plot], color=f"C{i}", alpha=0.5)
            if not is_dmn:
                ax.axhline(y=index[gaussian_i][metric_plot], color=f'C{i}', linestyle='--')
        ax.set_title(f"{gaussian_i} using Kraskov method")
        ax.set_xscale('log')
        ax.set_xlim(index_data[0], index_data[-1])
        ax.grid()
        ax.set_xlabel("N° data per sampling")
        ax.set_ylabel("value")
        fig.savefig(f"../results/plots/{metrics_plot}_{gaussian_i}.pdf")
    except:
        print("probably an error with index_data")
        pass

In [ ]:
metric_plot = "O"
i = 0
for gaussian_i in sorted(os.listdir(results_folder)):
    if i % 3 == 0:
        i = 0
        fig, ax = plt.subplots(figsize=(10,5))
    mean = pd.DataFrame(gaussian_results[gaussian_i]["mean"], columns=["TC", "DTC", "O"], index=index_data)
    std = pd.DataFrame(gaussian_results[gaussian_i]["std"], columns=["TC", "DTC", "O"], index=index_data)
    mean = mean.rename(columns={metric_plot: gaussian_i})
    mean[[gaussian_i]].plot(ax=ax)
    ax.fill_between(mean.index, mean[gaussian_i]-3*std[metric_plot], mean[gaussian_i]+3*std[metric_plot], alpha=0.2)
    if not is_dmn:
        ax.axhline(y=index[gaussian_i][metric_plot], color=f"C{i}", linestyle='--')
    ax.set_xscale('log')
    ax.set_title(metric_plot)
    plt.xlim(10**2, 10**5)
    plt.grid(axis="x")
    #fig.savefig(f"../results/plots/groups/group_{metric_plot}_{gaussian_i}.png")
    i += 1